<a href="https://colab.research.google.com/github/Cassandramack97/APS360/blob/main/APS360BaseModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code fomr MIE370 project 1 for reference

In [ ]:
### YOUR CODE HERE ###
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.model_selection import RandomizedSearchCV
#from scipy.stats import randint

#RFClassifier = RandomForestClassifier()

#paramsdistRF = {'max_features': ['sqrt', 'log2', 0.1, 0.25, 0.5], 'n_estimators': randint(10, 200)}

#randSearchRF = RandomizedSearchCV(RFClassifier, param_distributions=paramsdistRF, cv=5, verbose=2, random_state=42, n_jobs=-1, n_iter=500)

#randSearchRF.fit(stand_training_x, y_train)

#bestScoreRF = randSearchRF.best_score_
#bestParamsRF = randSearchRF.best_params_

#printing satement like the example
#print(f"Best n_estimators: {bestParamsRF['n_estimators']}, Best Max Features: {bestParamsRF['max_features']}, Best Accuracy: {bestScoreRF}")


getting access to the data

In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Making sure that it can access it
#for this to work add a shortcut to the folder in your drive
!ls "/content/drive/MyDrive/APS360_Project"

"FMA Bullshit Data Cleaning'"	      tensors2_val     tensors_val3	     train_metadata3.csv
'Good Data Cleaning and Processing'   tensors_test     test2_metadata.csv    train_metadata.csv
 Michael_Full_Cleaned_Dataset.csv     tensors_test3    test_metadata3.csv    val2_metadata.csv
 songs_with_details.csv		      tensors_train    test_metadata.csv     val_metadata3.csv
 tensors2_test			      tensors_train3   tracks_features.csv   val_metadata.csv
 tensors2_train			      tensors_val      train2_metadata.csv


In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.preprocessing import StandardScaler


Actually extracting features from the waveform files

####Saving this code in case we need it but the files should be feature tensors now
####sample rate prolly needs to be validated and explaine or tuned

import librosa

def extract_mfcc_with_timing(waveform, sr=22050, n_mfcc=13):
  mfcc = librosa.feature.mfcc(y=waveform, sr=sr, n_mfcc=n_mfcc)
  return mfcc  # Shape will be (n_mfcc, num_frames)

def extract_chroma_with_timing(waveform, sr=22050):
  chroma = librosa.feature.chroma_stft(y=waveform, sr=sr)
  return chroma  # Shape will be (12, num_frames) for chroma

def extract_spectral_contrast_with_timing(waveform, sr=22050):
  spectral_contrast = librosa.feature.spectral_contrast(y=waveform, sr=sr)
  return spectral_contrast  # Shape will be (num_bands, num_frames)

def extract_tonnetz_with_timing(waveform, sr=22050):
  tonnetz = librosa.feature.tonnetz(y=waveform, sr=sr)
  return tonnetz  # Shape will be (4, num_frames) for tonnetz

def extract_zero_crossing_rate_with_timing(waveform):
  zero_crossing_rate = librosa.feature.zero_crossing_rate(y=waveform)
  return zero_crossing_rate  # Shape will be (1, num_frames)

def extract_features_with_timing(waveform, sr=22050):
    mfcc = extract_mfcc_with_timing(waveform, sr)
    chroma = extract_chroma_with_timing(waveform, sr)
    spectral_contrast = extract_spectral_contrast_with_timing(waveform, sr)
    tonnetz = extract_tonnetz_with_timing(waveform, sr)
    zero_crossing = extract_zero_crossing_rate_with_timing(waveform)
    
    # Stack features along the feature dimension (axis=0)
    features = np.vstack([mfcc, chroma, spectral_contrast, tonnetz, zero_crossing])
    
    #time steps is the first dimension now
    #(time steps, features)
    return features.T


rnn is designed to take the variable length tensors but the rf regressor isn't so we have to convert them into fixed length vectors

rf also cant deal w temporal information so aggregating features is the best way to retain all the info and get fixed length vectors

**None of this will run cus the files arent real and also we need to make sure that the yTrain data (training labels) are the right format and type**

In [ ]:
!pip install unidecode
from unidecode import unidecode

#folder path
tensorFolderPath = '/content/drive/MyDrive/APS360_Project/tensors_train'

for file in os.listdir(tensorFolderPath):

    normalizedName = unidecode(file)

    if file != normalizedName:

        originalFile = os.path.join(tensorFolderPath, file)
        normalizedFile = os.path.join(tensorFolderPath, normalizedName)

        #rename the file
        os.rename(originalFile, normalizedFile)
        print(f"Renamed: {file} -> {normalizedName}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 17.5 MB/s eta 0:00:00


In [ ]:
#folder path
#tensorFolderPath = '/content/drive/MyDrive/APS360_Project/tensors_train'

#feature tensors
trainData = [file for file in os.listdir(tensorFolderPath)]

#getting the labels
labelsDF = pd.read_csv('/content/drive/MyDrive/APS360_Project/train_metadata.csv')

#removing the accents from the labels
labelsDF[labelsDF.columns[0]] = labelsDF[labelsDF.columns[0]].apply(unidecode)

#making sure the labels match the files
matchingLabelsDF = labelsDF[labelsDF[labelsDF.columns[0]].isin(trainData)]

#extracting the features and converting them to a numpy array for the rf
yTrain = matchingLabelsDF.iloc[:, 1:].values


In [ ]:
yTrain.shape

(140, 11)

For debugging to see what files/labels weren't matching

In [ ]:
missing_files = [tensorName for tensorName in trainData if tensorName not in matchingLabelsDF[matchingLabelsDF.columns[0]].values]
print(f"Number of missing files: {len(missing_files)}")
print("First few missing files:", missing_files[:10])

Number of missing files: 0
First few missing files: []


In [ ]:
missing_labels = [tensorName for tensorName in matchingLabelsDF[matchingLabelsDF.columns[0]].values if tensorName not in trainData]
print(f"Number of missing labels: {len(missing_labels)}")
print("First few missing labels:", missing_labels[:10])

Number of missing labels: 0
First few missing labels: []


In [ ]:
#convert tensors
def convertToVector(tensor):

  #we have to aggregate the features over time
  #so gotta get the std dev and mean
  meanFeatures = np.mean(tensor, axis=0)
  stdFeatures = np.std(tensor, axis=0)

  #concatenate into a single feature vector that can be taken in by the rf
  return np.concatenate((meanFeatures, stdFeatures))

#to store the vectors
vectorList = []

#converting to the vectors for rf
for tensorPath in trainData:

  fullPath = os.path.join(tensorFolderPath, tensorPath)

  tensor = np.load(fullPath)

  vectorList.append(convertToVector(tensor))

xTrain = np.array(vectorList)

In [ ]:
xTrain

array([[-4.70133055e+01,  7.61315917e+01, -2.63300001e+01, ...,
         4.86488789e-02,  4.20257705e-02,  3.36938167e-02],
       [-2.55471597e+02,  8.26372579e+01, -8.42534467e+00, ...,
         6.26157404e-02,  7.16114163e-02,  5.83521786e-02],
       [-5.25387701e+00,  8.14500267e+01, -2.93795480e+01, ...,
         2.15557996e-02,  2.05794249e-02,  4.40538322e-02],
       ...,
       [-1.59306451e+02,  1.01599863e+02,  3.78970325e+00, ...,
         2.76581058e-02,  2.98427170e-02,  4.66764535e-02],
       [-2.77628002e+02,  1.33452131e+02, -1.68527997e+01, ...,
         6.12851580e-02,  6.93194129e-02,  3.19360539e-02],
       [-3.06779555e+02,  1.42870791e+02, -2.10423597e-01, ...,
         6.28015282e-02,  6.43143522e-02,  3.06264456e-02]])

For validation and testing now

In [ ]:
#folder path
valTensorFolderPath = '/content/drive/MyDrive/APS360_Project/tensors_val'

#feature tensors
valData = [file for file in os.listdir(valTensorFolderPath)]

for file in os.listdir(valTensorFolderPath):

    normalizedName = unidecode(file)

    if file != normalizedName:

        originalFile = os.path.join(valTensorFolderPath, file)
        normalizedFile = os.path.join(valTensorFolderPath, normalizedName)

        #rename the file
        os.rename(originalFile, normalizedFile)
        print(f"Renamed: {file} -> {normalizedName}")

In [ ]:
#getting the labels
valLabelsDF = pd.read_csv('/content/drive/MyDrive/APS360_Project/val_metadata.csv')

#removing the accents from the labels
valLabelsDF[valLabelsDF.columns[0]] = valLabelsDF[valLabelsDF.columns[0]].apply(unidecode)

#making sure the labels match the files
matchingValLabelsDF = valLabelsDF[valLabelsDF[valLabelsDF.columns[0]].isin(valData)]

#extracting the features and converting them to a numpy array for the rf
yVal = matchingValLabelsDF.iloc[:, 1:].values

In [ ]:
#to store the vectors
valVectorList = []

#converting to the vectors for rf
for tensorPath in valData:

  fullValPath = os.path.join(valTensorFolderPath, tensorPath)

  tensor = np.load(fullValPath)

  valVectorList.append(convertToVector(tensor))

xVal = np.array(valVectorList)

In [ ]:
#folder path
testTensorFolderPath = '/content/drive/MyDrive/APS360_Project/tensors_test'

#feature tensors
testData = [file for file in os.listdir(testTensorFolderPath)]

for file in os.listdir(testTensorFolderPath):

    normalizedName = unidecode(file)

    if file != normalizedName:

        originalFile = os.path.join(testTensorFolderPath, file)
        normalizedFile = os.path.join(testTensorFolderPath, normalizedName)

        #rename the file
        os.rename(originalFile, normalizedFile)
        print(f"Renamed: {file} -> {normalizedName}")

In [ ]:
#getting the labels
testLabelsDF = pd.read_csv('/content/drive/MyDrive/APS360_Project/test_metadata.csv')

#removing the accents from the labels
testLabelsDF[testLabelsDF.columns[0]] = testLabelsDF[testLabelsDF.columns[0]].apply(unidecode)

#making sure the labels match the files
matchingTestLabelsDF = testLabelsDF[testLabelsDF[testLabelsDF.columns[0]].isin(testData)]

#extracting the features and converting them to a numpy array for the rf
yTest = matchingTestLabelsDF.iloc[:, 1:].values

In [ ]:
#to store the vectors
testVectorList = []

#converting to the vectors for rf
for tensorPath in testData:

  fullTestPath = os.path.join(testTensorFolderPath, tensorPath)

  tensor = np.load(fullTestPath)

  testVectorList.append(convertToVector(tensor))

xTest = np.array(testVectorList)

actual model

In [ ]:
#defining the output labels
outputLabels = ["valence", "liveness", "time_signature", "energy", "speechiness",
                 "instrumentalness", "acousticness", "tempo", "loudness", "danceability", "key"]

#define the regressor
#n_estimators is a hyperparam for the number of decision trees used
#more trees reduces var but inc time
#other hyperparams: max_depth, max_features, min_samples_split, max_samples_split, bootstrap
rfRegressor = RandomForestRegressor(n_estimators = 100, random_state = 42)

#fit
#rfRegressor.fit(xTrain, yTrain)

In [ ]:
#hyper parameter tuning
paramsdistRF = {'max_features': ['sqrt', 'log2', 0.1, 0.25, 0.5, 0.75, 1.0], 'n_estimators': randint(50, 1000), 'max_depth': range(10, 100), 'bootstrap': [True, False]}

mseScorer = make_scorer(mean_squared_error, greater_is_better=False)

randSearchRF = RandomizedSearchCV(rfRegressor, param_distributions=paramsdistRF, cv=5, verbose=2, random_state=42, n_jobs=-1, n_iter=500, scoring = mseScorer)

randSearchRF.fit(xTrain, yTrain)

bestParamsRF = randSearchRF.best_params_
bestScoreRF = randSearchRF.best_score_

# Convert negative MSE back to positive
bestMSE = -bestScoreRF

print(f"Best n_estimators: {bestParamsRF['n_estimators']}, Best Max Features: {bestParamsRF['max_features']}, Best MSE: {bestMSE}, Best Depth: {bestParamsRF['max_depth']}, Best Bootstrap: {bestParamsRF['bootstrap']}")


Fitting 5 folds for each of 500 candidates, totalling 2500 fits
Best n_estimators: 269, Best Max Features: log2, Best MSE: 125.536297470898, Best Depth: 11, Best Bootstrap: True


Validation data

In [ ]:
bestRFRegressor = RandomForestRegressor(n_estimators=bestParamsRF['n_estimators'], max_features=bestParamsRF['max_features'], max_depth=bestParamsRF['max_depth'], bootstrap=bestParamsRF['bootstrap'], random_state=42)

#fit
bestRFRegressor.fit(xTrain, yTrain)

#run on validation
valPreds = bestRFRegressor.predict(xVal)

#compute MSE for the multiple outputs
featureMSEs = mean_squared_error(yVal, valPreds, multioutput='raw_values')

print("MSE for each feature:")
for i, feature in enumerate(outputLabels):
    print(f"{feature}: {featureMSEs[i]}")

MSE for each feature:
valence: 0.0605951786010522
liveness: 0.06574819767100504
time_signature: 0.10353868637099022
energy: 0.0794116772372024
speechiness: 0.0016918543865124262
instrumentalness: 0.14458226564016127
acousticness: 0.12319602276268696
tempo: 841.6317075476669
loudness: 41.14810815200283
danceability: 0.02523081208832879
key: 14.723000272332996


Testing Data

In [ ]:
bestRFRegressor = RandomForestRegressor(n_estimators=bestParamsRF['n_estimators'], max_features=bestParamsRF['max_features'], max_depth=bestParamsRF['max_depth'], bootstrap=bestParamsRF['bootstrap'], random_state=42)

#fit
bestRFRegressor.fit(xTrain, yTrain)

#run on validation
testPreds = bestRFRegressor.predict(xTest)

#compute MSE for the multiple outputs
featureMSEs = mean_squared_error(yTest, testPreds, multioutput='raw_values')

print("MSE for each feature:")
for i, feature in enumerate(outputLabels):
    print(f"{feature}: {featureMSEs[i]}")